# Figures

In [ ]:
import re
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import pickle
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

def extract_number_eis(file_name):
    match = re.search(r'eis_(\d+)_', file_name)
    return int(match.group(1)) if match else -1  # Return -1 if no match found

def extract_number_rest(file_name):
    match = re.search(r'rest_(\d+)_', file_name)
    return int(match.group(1)) if match else -1  # Return -1 if no match found

## Figure 1

In [ ]:
ftsz = 14
folder_path = "data_120s\train\sample"

# Rest
fig, axes = plt.subplots(3, 1, figsize=(2.5, 4.6))
cmap = cm.coolwarm
files = [f for f in os.listdir(folder_path) if f.startswith('rest') and f.endswith('.pkl')]
files.sort(key=extract_number_rest)
for i, file_name in enumerate(files):
    file_path = os.path.join(folder_path, file_name)
    rest = pd.read_pickle(file_path)
    color = cmap(i / len(files))
    axes[0].plot(rest['time'], rest['<I>/mA'],'.' , color=color, alpha=0.7)
    axes[0].set_ylabel('Current', fontsize=ftsz)
    axes[1].plot(rest['time'], rest['Ecell/V'], color=color, alpha=0.7)
    axes[1].set_ylabel('Voltage', fontsize=ftsz)
    axes[2].plot(rest['time'], rest['Temperature/°C'], color=color, alpha=0.7)
    axes[2].set_xlabel('time', fontsize=ftsz)
    axes[2].set_ylabel('Temperature', fontsize=ftsz)

plt.tight_layout()
plt.show()

# EIS
plt.figure(figsize=(3.2,3))
files = [f for f in os.listdir(folder_path) if f.startswith('eis') and f.endswith('.pkl')]
files.sort(key=extract_number_eis)
cmap = plt.get_cmap('coolwarm')
for i, file_name in enumerate(files):
    file_path = os.path.join(folder_path, file_name)
    eis = pd.read_pickle(file_path)
    color = cmap(i / len(files))
    plt.plot(eis['Re(Z)/mOhm'].iloc[1:], eis['-Im(Z)/mOhm'].iloc[1:], color=color, alpha=0.7)
plt.xlabel(r'$Z_{\rm re}$ (mΩ)', fontsize=ftsz)
plt.ylabel(r'$-Z_{\rm im}$ (mΩ)', fontsize=ftsz)
plt.tight_layout()
plt.show()

# DRT
ftsz = 16
pkl_path = r"paper_figure\drt\drt_results_S016.pkl"
with open(pkl_path, 'rb') as f:
    results = pickle.load(f)
num_eis = len(results["R_inf_DRT_meas"])
plt.figure(figsize=(4,3))
cmap = plt.get_cmap('coolwarm')
for i in range(num_eis):
    color = cmap(i / num_eis)
    plt.semilogx(results["tau_vec"], results["gamma_DRT_meas"][i], '-', c=color, alpha=0.8)
plt.xlabel('τ (s)', fontsize=ftsz)
plt.ylabel('γ(τ) (mΩ)', fontsize=ftsz)
plt.tight_layout()
plt.show()


## Figure 2b

In [ ]:
file_path1 = "cap_no_kit.pkl"
with open(file_path1, 'rb') as f:
    dfS_nk = pickle.load(f)

file_path2 = "cap_L.pkl"
with open(file_path2, 'rb') as f:
    dfS_L = pickle.load(f)

file_path3 = "cap_S.pkl"
with open(file_path3, 'rb') as f:
    dfS_S = pickle.load(f)

folder_path4 = "pkl_cyEq"

with open(file_path3, 'rb') as f:
    dfS_T = pickle.load(f)

plt.figure(figsize=(5, 4))

cmapS = plt.cm.get_cmap('Reds')
cmapL = plt.cm.get_cmap('Purples')
cmapK = plt.cm.get_cmap('Blues')
cmapT = plt.cm.get_cmap('Greens')

plt.plot([], [], '.-', color=cmapS(0.5), linewidth=1.5, markersize=4, label='Dataset S')
plt.plot([], [], '.-', color=cmapL(0.5), linewidth=1.5, markersize=4, label='Dataset L')
plt.plot([], [], '.-', color=cmapK(0.5), linewidth=1.5, markersize=4, label='Dataset K')
plt.plot([], [], '.-', color=cmapT(0.5), linewidth=1.5, markersize=4, label='Dataset T')

for i, (cell_id, df) in enumerate(dfS_S.items()):
    df.loc[df['Capacity']==0,'Capacity'] = np.nan
    df = df.dropna()
    color = cmapS(0.3 + 0.6 * i / len(dfS_S))
    plt.plot(df['RPT'], df['Capacity'], '.-', 
             color=color, linewidth=1.5, markersize=4, alpha=0.8, label="_nolegend_")

for i, (cell_id, df) in enumerate(dfS_L.items()):
    df.loc[df['Capacity']==0,'Capacity'] = np.nan
    df = df.dropna()
    color = cmapL(0.3 + 0.6 * i / len(dfS_L))
    plt.plot(df['RPT'], df['Capacity'], '.-', 
             color=color, linewidth=1.5, markersize=4, alpha=0.8, label="_nolegend_")

for i, (cell_id, df) in enumerate(dfS_nk.items()):
    df.loc[df['Capacity']==0,'Capacity'] = np.nan
    df = df.dropna()
    color = cmapK(0.3 + 0.6 * i / len(dfS_nk))
    plt.plot(df['RPT'], df['Capacity'], '.-', 
             color=color, linewidth=1.5, markersize=4, alpha=0.8, label="_nolegend_")

files = [f for f in os.listdir(folder_path4) if f.endswith('.pkl')]
n_files = len(files)
for i, file in enumerate(files):
    file_path = os.path.join(folder_path4, file)
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
    color = cmapT(0.3 + 0.6 * i / n_files)
    plt.plot(df['cap'], '.-', 
             color=color, linewidth=1.5, markersize=4, alpha=0.8, label="_nolegend_")

plt.xlabel('RPTs', fontsize=16)
plt.ylabel('Capacity (Ah)', fontsize=16)
plt.gca().set_facecolor('#f9f9f9')
plt.legend(fontsize=14)
plt.show()

## Figure 2c-e

In [ ]:
def extract_number_eis(filename):
    match = re.search(r'eis_(\d+)', filename)
    if match:
        return int(match.group(1))
    return 0

def eis_3D_plot(folder_path, n_RPT_max, save_path):
    ftsz = 30
    tksz = 20
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    cmap = plt.get_cmap('coolwarm')
    n_last = 0
    for folder_idx, folder in enumerate(subfolders):
        cell_folder_path = os.path.join(folder_path, folder)
        files = [f for f in os.listdir(cell_folder_path) if f.startswith('eis') and f.endswith('.pkl')]
        if not files:
            continue
        files.sort(key=extract_number_eis)

        for i, file_name in enumerate(files):
            file_path = os.path.join(cell_folder_path, file_name)
            eis = pd.read_pickle(file_path)
            z_re = eis['Re(Z)/mOhm'].iloc[1:].values
            z_im = eis['-Im(Z)/mOhm'].iloc[1:].values
            x_axis = n_last + i
            x_array = np.full_like(z_re, fill_value=x_axis, dtype=float)
            color = cmap(i / n_RPT_max)
            ax.plot(x_array, z_re, z_im, color=color, alpha=0.7, linewidth=1.5)
        n_last = n_last + i + 1
    ax.set_ylabel(r'$Z_{\rm re}$ (mΩ)', fontsize=ftsz, labelpad=15)
    ax.set_zlabel(r'$-Z_{\rm im}$ (mΩ)', fontsize=ftsz, labelpad=15)
    ax.set_xlabel('Samples', fontsize=ftsz, labelpad=20)
    ax.tick_params(axis='both', which='major', labelsize=tksz)
    ax.view_init(elev=30, azim=-30)

    norm = mpl.colors.Normalize(vmin=0, vmax=n_RPT_max)
    sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, shrink=0.6, pad=0.11)
    cbar.set_label('RPT No.', fontsize=ftsz, labelpad=10)
    cbar.ax.tick_params(labelsize=tksz)
    cbar.outline.set_visible(False)

    plt.tight_layout()
    plt.show()

def extract_number_rest(filename):
    match = re.search(r'rest_(\d+)', filename)
    if match:
        return int(match.group(1))
    return 0

ftsz = 30
tksz = 20

def rest_3D_plot(folder_path, n_RPT_max, save_path):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    cmap = plt.get_cmap('coolwarm')
    
    n_last = 0
    for folder_idx, folder in enumerate(subfolders):
        cell_folder_path = os.path.join(folder_path, folder)
        files = [f for f in os.listdir(cell_folder_path) if f.startswith('rest') and f.endswith('.pkl')]
        if not files:
            continue
        files.sort(key=extract_number_rest)
        for i, file_name in enumerate(files):
            file_path = os.path.join(cell_folder_path, file_name)
            rest = pd.read_pickle(file_path)
            rest.loc[rest['time'] < 0] = np.nan
            t = rest['time'].values
            volt = rest['Ecell/V'].values
            x_axis = n_last + i
            x_array = np.full_like(t, fill_value=x_axis, dtype=float)
            color = cmap(i / n_RPT_max)
            ax.plot(x_array, t, volt, color=color, alpha=0.7, linewidth=1.5)
        n_last = n_last + i + 1

    ax.set_ylabel('time (s)', fontsize=ftsz, labelpad=15)
    ax.set_zlabel('Voltage (V)', fontsize=ftsz, labelpad=15)
    ax.set_xlabel('Samples', fontsize=ftsz, labelpad=20)
    ax.tick_params(axis='both', which='major', labelsize=tksz)
    ax.view_init(elev=20, azim=-30)

    norm = mpl.colors.Normalize(vmin=0, vmax=n_RPT_max)
    sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, shrink=0.6, pad=0.11)
    cbar.set_label('RPT No.', fontsize=ftsz, labelpad=10)
    cbar.ax.tick_params(labelsize=tksz)
    cbar.outline.set_visible(False)

    plt.tight_layout()
    plt.show()

## Figure 3 EIS_prediction

In [ ]:
from matplotlib.lines import Line2D

def Z_re_im_get(data):
    meas_mag = data['meas_mag'] * 1000
    meas_ph = data['meas_ph']
    pred_mag = data['pred_mag'] * 1000
    pred_ph = data['pred_ph']
    frequency = data['freq']
    theta_meas_rad = np.deg2rad(data['meas_ph'])
    theta_pred_rad = np.deg2rad(data['pred_ph'])
    meas_real = data['meas_mag'] * np.cos(theta_meas_rad) * 1000   # mΩ
    meas_imag = -data['meas_mag'] * np.sin(theta_meas_rad) * 1000
    pred_real = data['pred_mag'] * np.cos(theta_pred_rad) * 1000
    pred_imag = -data['pred_mag'] * np.sin(theta_pred_rad) * 1000
    return frequency,meas_mag,meas_ph,pred_mag,pred_ph,meas_real,meas_imag,pred_real,pred_imag

def peis_3D_plot(folder_path, save_path, n_max):
    ftsz = 30
    tksz = 20
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    cmap_1 = plt.get_cmap('Purples')
    cmap_2 = plt.get_cmap('Oranges')

    n_last = 0
    for folder_idx, folder in enumerate(subfolders):
        cell_folder_path = os.path.join(folder_path, folder)
        files = [f for f in os.listdir(cell_folder_path) if f.startswith('predicted_eis') and f.endswith('.pkl')]
        if not files:
            continue
        files.sort(key=extract_number_eis)
        for i, file_name in enumerate(files):
            file_path = os.path.join(cell_folder_path, file_name)
            data = pd.read_pickle(file_path)
            frequency,meas_mag,meas_ph,pred_mag,pred_ph,meas_real,meas_imag,pred_real,pred_imag = Z_re_im_get(data)
            x_axis = n_last + i
            x_array = np.full_like(meas_real, fill_value=x_axis, dtype=float)
            i_rpt = extract_number_eis(file_name)
            c1 = cmap_1(np.linspace(0.2, 0.8, n_max))[i_rpt]
            c2 = cmap_2(np.linspace(0.2, 0.8, n_max))[i_rpt]
            ax.plot(x_array, pred_real, pred_imag, '-', color=c1, alpha=0.9, linewidth=3)
            ax.plot(x_array, meas_real, meas_imag, '--', color=c2, alpha=0.9, linewidth=3)
        n_last = n_last + i + 1
        
    ax.set_ylabel(r'$Z_{\rm re}$ (mΩ)', fontsize=ftsz, labelpad=15)
    ax.set_zlabel(r'$-Z_{\rm im}$ (mΩ)', fontsize=ftsz, labelpad=15)
    ax.set_xlabel('Samples', fontsize=ftsz, labelpad=20)
    ax.tick_params(axis='both', which='major', labelsize=tksz)
    ax.view_init(elev=45, azim=-20)

    legend_elements = [
        Line2D([0], [0], linestyle='-', color=cmap_1(0.6), label='Predicted', linewidth=3),
        Line2D([0], [0], linestyle='--', color=cmap_2(0.6), label='Measured', linewidth=3),
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=ftsz-4, ncols=2)

    plt.tight_layout()
    plt.show()

## Figure 3 fit

In [ ]:
from sklearn.metrics import r2_score
ms = 15
ftsz = 14

def extract_number_eis(file_name):
    match = re.search(r'eis_(\d+)_', file_name)
    return int(match.group(1)) if match else -1

def Z_re_im_get(data):
    meas_mag = data['meas_mag'] * 1000  #mΩ
    meas_ph = data['meas_ph']
    pred_mag = data['pred_mag'] * 1000  #mΩ
    pred_ph = data['pred_ph']
    frequency = data['freq']
    theta_meas_rad = np.deg2rad(data['meas_ph'])
    theta_pred_rad = np.deg2rad(data['pred_ph'])
    meas_real = data['meas_mag'] * np.cos(theta_meas_rad) * 1000   # mΩ
    meas_imag = -data['meas_mag'] * np.sin(theta_meas_rad) * 1000
    pred_real = data['pred_mag'] * np.cos(theta_pred_rad) * 1000
    pred_imag = -data['pred_mag'] * np.sin(theta_pred_rad) * 1000
    return frequency,meas_mag,meas_ph,pred_mag,pred_ph,meas_real,meas_imag,pred_real,pred_imag

def fit_magn_phase_scatter(cell_path, ax_magnitude, ax_phase, cmap_k):
    predict_eis_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl")]
    sorted_predict_eis_files = sorted(predict_eis_files, key=lambda x: extract_number_eis(x))
    num_plots = len(sorted_predict_eis_files)
    for i, seg in enumerate(sorted_predict_eis_files):
        with open(os.path.join(cell_path, seg), 'rb') as f:
            data = pickle.load(f)
        frequency,meas_mag,meas_ph,pred_mag,pred_ph,meas_real,meas_imag,pred_real,pred_imag = Z_re_im_get(data)      
        c_1 = cmap_k(np.linspace(0.1, 0.9, num_plots))[i]
        ax_magnitude.scatter(meas_mag, pred_mag, edgecolor='none', facecolor=c_1, alpha=0.8, s=ms)
        ax_phase.scatter(meas_ph, pred_ph, edgecolor='none', facecolor=c_1, alpha=0.8, s=ms)


## Figure 3 Heatmap

In [ ]:
import seaborn as sns

def result_heatmap(folder, save_path, filenames, err_index, cbar_label):
    ftsz = 14
    data = {}
    for fname in filenames:
        filepath = os.path.join(folder, fname)
        df = pd.read_excel(filepath, sheet_name='Sheet1')
        df_subset = df.set_index('RPT')[err_index]
        data[os.path.splitext(fname)[0]] = df_subset

    df_all = pd.DataFrame.from_dict(data, orient='index')
    plt.figure(figsize=(7.5, 3.5))
    ax = sns.heatmap(df_all, cmap='coolwarm', annot=False, cbar_kws={'label': cbar_label})
    ax.set_xlabel('RPT No.', fontsize=ftsz)
    ax.set_ylabel('Test Cell No.', fontsize=ftsz)
    cbar = ax.collections[0].colorbar
    cbar.ax.yaxis.label.set_size(ftsz)
    plt.tight_layout()
    plt.show()


## Figure 3 Violin_plot

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def load_mag_phase_errors_all_subfolders(root_path):
    mag_errors = []
    ph_errors = []
    freqs = None
    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if file.endswith('.pkl'):
                file_path = os.path.join(dirpath, file)
                with open(file_path, 'rb') as f:
                    data = pickle.load(f)
                if freqs is None:
                    freqs = data['freq']
                meas_mag = data['meas_mag'] * 1000  # mΩ
                pred_mag = data['pred_mag'] * 1000
                mag_error = meas_mag - pred_mag
                meas_ph = data['meas_ph']
                pred_ph = data['pred_ph']
                ph_error = meas_ph - pred_ph
                mag_errors.append(mag_error)
                ph_errors.append(ph_error)
    return freqs, np.array(mag_errors), np.array(ph_errors)

def to_violin_df(freqs, errors, error_name, unit):
    df = pd.DataFrame(errors, columns=freqs)
    df_long = df.melt(var_name="Frequency (Hz)", value_name=f"{error_name} ({unit})")
    df_long = df_long.sort_values(by="Frequency (Hz)").reset_index(drop=True)
    return df_long

def assign_decade(freq):
    return int(np.floor(np.log10(freq)))

_superscript_map = {
    "-": "⁻",
    "0": "⁰",
    "1": "¹",
    "2": "²",
    "3": "³",
    "4": "⁴",
    "5": "⁵",
    "6": "⁶",
    "7": "⁷",
    "8": "⁸",
    "9": "⁹"
}

def to_superscript(n):
    return ''.join(_superscript_map.get(c, '') for c in str(n))

def format_decade_label(decade):
    high_exp = to_superscript(decade + 1)
    low_exp = to_superscript(decade)
    return f"10{high_exp}–10{low_exp}"

def plot_violin_by_decade(df_long, value_col, save_path):
    df_long["Frequency (Hz)"] = df_long["Frequency (Hz)"].astype(float)
    df_long["Decade"] = df_long["Frequency (Hz)"].apply(assign_decade)
    unique_decades = sorted(df_long["Decade"].unique(), reverse=True)
    decade_labels = {d: format_decade_label(d) for d in unique_decades}
    df_long["Decade Label"] = df_long["Decade"].map(decade_labels)
    df_long["Decade Label"] = pd.Categorical(
        df_long["Decade Label"],
        categories=[decade_labels[d] for d in unique_decades],
        ordered=True
    )
    plt.figure(figsize=(8, 6))
    ax = sns.violinplot(
        data=df_long,
        x="Decade Label",
        y=value_col,
        inner="quartile",
        palette="coolwarm_r"
    )
    ax.tick_params(axis='x', labelrotation=0, labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_xlabel("Frequency Range (Hz)", fontsize=20)
    ax.set_ylabel(value_col, fontsize=20)
    plt.tight_layout()
    plt.show()

